In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/colab_projects/nlp_from_dls/3_HW_Simple_embeddings/

/content/drive/MyDrive/colab_projects/nlp_from_dls/3_HW_Simple_embeddings


In [ ]:
%ls

'Copy of [homework]simple_embeddings.ipynb'
 data/
'SO_vectors_200.bin?download=1'
 stackoverflow_similar_questions.zip


<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
# try next cell one first, maybe it's already loaded
#
# !wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

In [ ]:
%%time

from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

CPU times: user 30.1 s, sys: 4.42 s, total: 34.5 s
Wall time: 37.8 s


Больше инфы про [KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) в документации [gensim](https://radimrehurek.com/gensim/auto_examples/index.html#documentation).



#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
# have a look at the first 20 values
print(wv_embeddings['dog'][:20])

[ 0.6851772  -1.2778991  -0.41913974  1.3623164  -3.1675398   0.09950767
  0.6402681  -1.1245339  -0.6699619  -0.6998852   0.4936771  -0.40500194
 -3.0706816  -2.2809966   0.85798043  2.7093108   0.3492745  -0.03494101
 -0.22330493  1.2290467 ]


In [ ]:
print(type(wv_embeddings.index2word))
print(f"Num of words: {len(wv_embeddings.index2word)}")

<class 'list'>
Num of words: 1787145


In [ ]:
# examples of words
print('Words', 5*' ', 'First 10 (out of 200) values in the embedding')
print('-'*83)

for i, word in enumerate(wv_embeddings.index2word):
    if i < 20:
        print(word, (10-len(word)) * ' ', wv_embeddings[word][:10].round(3))
    else:
        break

Words       First 10 (out of 200) values in the embedding
-----------------------------------------------------------------------------------
use         [-1.325  2.425 -0.49   2.941 -3.2    1.971  2.733  0.68  -0.77  -2.533]
code        [ 0.774 -0.671  0.139  3.103  0.163  2.15   0.1   -1.269 -0.649 -1.672]
using       [-0.295  1.666 -1.025  2.713 -2.156 -0.339  1.496  0.294  1.283 -2.021]
like        [-0.691 -0.583 -1.066  1.855 -0.94  -0.032  3.284 -0.216 -0.161 -1.501]
will        [ 0.24  -2.395  0.88   4.439 -1.777  1.392  2.478  1.205 -1.943 -0.935]
want        [ 1.849  0.291  0.309  3.915 -2.856  1.17   2.392  1.68  -0.559 -0.628]
need        [ 0.49   0.097  0.357  2.639 -2.237  0.56   1.019  0.883 -0.407 -0.15 ]
get         [-0.021 -1.979 -1.851  2.119  0.541  0.213  0.519  2.215  0.031 -2.57 ]
file        [ 1.465  2.534 -2.501  3.009 -0.668  3.837  1.379  0.758 -0.158  1.483]
one         [-0.81   1.657 -1.19   1.862 -1.146  0.499  1.678 -0.369 -0.545 -0.86 ]
just        [ 2.13

In [ ]:
type(wv_embeddings.vocab), len(wv_embeddings.vocab)

(dict, 1787145)

In [ ]:
wv_embeddings.vocab['job']

Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [ ]:
# method most_simmilar


# idea from: https://github.com/vefstathiou/SO_word2vec
try:
    result = wv_embeddings.most_similar('dog', topn=5)
except KeyError as e:
        print(e)


# print numbers and words, not really interested in similarities
for i, (sim_word, similarity) in enumerate(result):
    print(i, sim_word, (10-len(sim_word))*' ', similarity)

0 animal      0.8564180135726929
1 dogs        0.7880867123603821
2 mammal      0.7623804807662964
3 cats        0.7621253728866577
4 animals     0.760793924331665


`cat`не входит. Но `cats` на 4 месте (считая с 1).

Странно, что "dogs" дальше от 'dog', чем 'animal'. Число, видимо, играет роль.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopWords = set(stopwords.words('english'))
print(stopWords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{'do', 'between', "you'd", 'having', 'hasn', "you're", 'and', "weren't", 'wouldn', 'so', 'very', 've', 'why', 'its', 'few', 'than', "that'll", 'off', 'don', 'these', 'hers', 'on', "you've", 'what', "wouldn't", 'then', 'theirs', 'had', 's', 'too', 'him', 'our', 'haven', "needn't", "doesn't", 'was', 'under', 'in', 'didn', 'myself', 'are', "wasn't", 'until', "don't", 'themselves', 'further', 'other', 'as', 'the', 'while', 'that', 'below', 'most', "haven't", "hasn't", 'or', "hadn't", 'd', 'each', 're', 'they', "shan't", 'but', 't', 'm', "isn't", 'if', 'there', 'his', 'about', 'were', 'doesn', 'with', 'to', 'am', 'ours', 'has', 'up', 'those', 'be', 'my', 'this', 'who', 'her', 'before', 'i', 'during', 'for', 'll', 'should', 'out', 'mustn', 'some', 'y', 'down', 'nor', 'over', 'just', 'more', 'whom', 'here', "it's", 'their', 'when', 'she', 'above', 'herself', 'through', 'ma', 

In [ ]:
import spacy
nlp = spacy.load('en')

In [ ]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        # modifications
        #   Do not split +# from words
        #   use .lower()
        tokens = re.findall('\w+[+#]*', text.lower())
        tokens = [t for t in tokens if t not in stopWords]

        return tokens


my_tokenizer = MyTokenizer()
print(my_tokenizer.tokenize("Tolstoy began writing War and Peace in 1863, the year that he finally married and settled down at his country estate. He wrote it in c++ instead of c#."), '\n')

['tolstoy', 'began', 'writing', 'war', 'peace', '1863', 'year', 'finally', 'married', 'settled', 'country', 'estate', 'wrote', 'c++', 'instead', 'c#'] 



Our custom tokenizer **takes out** the spaces and punctuation and **uses them as separators**.

In [ ]:
from nltk.tokenize import WordPunctTokenizer

nltk_tokenizer = WordPunctTokenizer()
print(nltk_tokenizer.tokenize("Tolstoy began writing War and Peace in 1863, the year that he finally married and settled down at his country estate. He wrote it in c++ instead of c#."), '\n')

['Tolstoy', 'began', 'writing', 'War', 'and', 'Peace', 'in', '1863', ',', 'the', 'year', 'that', 'he', 'finally', 'married', 'and', 'settled', 'down', 'at', 'his', 'country', 'estate', '.', 'He', 'wrote', 'it', 'in', 'c', '++', 'instead', 'of', 'c', '#.'] 



The `WordPunctTokenizer` does not take out the punctuation. Might be useful for getting to know where the sentences split, but it splits the symbols from languages, e.g. C++, and C# will be stripped from the symbols.

In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    tokens = tokenizer.tokenize(question)
    res = np.zeros(dim)

    known_tokens = 0
    for token in tokens:
        
        if token in embeddings:
            assert(len(embeddings[token]) == dim)
            res += embeddings[token]
            known_tokens += 1
    
    # none of the tokens are in the embeddings
    #   => res stayed as it was initialized (np.zeros(dim))
    if known_tokens == 0:
        return res
    else:
        return res / known_tokens

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
sample_text = 'I love neural networks'

print('With custom tokenizer:', question_to_vec(question=sample_text, embeddings=wv_embeddings, tokenizer=my_tokenizer)[2].round(2))
print('With nltk tokenizer:', question_to_vec(question=sample_text, embeddings=wv_embeddings, tokenizer=nltk_tokenizer)[2].round(2))

With custom tokenizer: -1.29
With nltk tokenizer: -1.29


Ответ: -1.29

Ответ одинаковый потому что в этом предлложении нет ни пунктуации, ни стоп-слов, поэтому tokenizer-ы работают одинаково.

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

----

**Мои комментарии:**

1. Метрику Hits@K мы хотим приблизить как можно ближе к 1. Это достижимо когда для каждого $i$ выполняется $rank\_q_i^{'} \le K$. Тогда: $$\text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K] = \frac{1}{N}\sum_{i=1}^N 1 = 1$$

2. Идеально, если дупликат всегда на первом месте. Тогда $\forall K \ge 0$ выполняется Hits@K = 1

3. Метрика DCG@K имеет тот же смымл: чем ближе к 1 (снизу) тем лучше. Но достичь идеального результата труднее. Теперь нам не достаточно выполнить $\forall i : rank\_q_i^{'} \le K$, чтобы достичь  DCG@K=1. Теперь нужно $\forall i : rank\_q_i^{'} = 1$ потому что только в этом случае $$\frac{1}{\log_2(1+rank\_q_i^{'})} = \frac{1}{\log_2(1+1)} = \frac{1}{1} = 1$$. В противном случае, DCG@K < 1.

4. Значения обеих метрик ограничены снизу 0. 0 достигается, если индикатор равен нулю для всех $i \in \{1,.., N\}$

----

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Ответ: 1

Объясненние: Предположим, что для всех $i \in \{1,.., N\}$ выполняется $rank\_q_i^{'} = 2$. Тогда DCG@1 = 0, а Hits@47=1. Мы достигли максимальной возможной разницы, т.к. обе метрики оограничены интервалом [0, 1].

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

* [K = 10] $\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}} \sim 3.3$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """

    # inner brackets: compute all indicators
    hits_value = np.mean(np.array(dup_ranks) <= k)
    return hits_value    

In [ ]:
# check if it works
nums = (np.array([1, 2, 3, 4, 5]) <= 3).astype(np.int)
denoms = 1 / np.log2(1 + np.array([1, 2, 3, 4, 5]))

np.mean(nums * denoms)

0.42618595071429155

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    dup_ranks_arr = np.array(dup_ranks)

    hit_indicators = (dup_ranks_arr <= k).astype(np.int)
    denoms = 1 / np.log2(1 + dup_ranks_arr)

    dcg_value = np.mean(denoms * hit_indicators)

    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2] # Carefull! Best rank is 1, not 0.

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[Download link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit) for a zip file.

You can just add a shortcut to your drive, or use the following command. In this case, there will be no need to mount the drive.

`!gdown --id '1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_'`

----

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [2]:
# try the next cell first to

# !unzip stackoverflow_similar_questions.zip #__MACOSX/\*# -d ./data/
# %rm -r __MACOSX/
# %rm data/.DS_Store

Downloading...
From: https://drive.google.com/uc?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_
To: /content/stackoverflow_similar_questions.zip
131MB [00:01, 117MB/s]


In [3]:
%ls

sample_data/  stackoverflow_similar_questions.zip


Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        q_data = line.split('\t')
        data.append(q_data)

    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
# question and its vector representation
q = 'a'
qv = [0, 1]

# candidates and their vector representations
cs = ['a', 'b', 'c', 'd']
cvs = [[0, 1], [1, 0], [0.2, 0.3], [0.5, 0.5]]

trio = list(zip(range(len(cs)), cs, cvs))
trio

[(0, 'a', [0, 1]),
 (1, 'b', [1, 0]),
 (2, 'c', [0.2, 0.3]),
 (3, 'd', [0.5, 0.5])]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
help(cosine_similarity)

Help on function cosine_similarity in module sklearn.metrics.pairwise:

cosine_similarity(X, Y=None, dense_output=True)
    Compute cosine similarity between samples in X and Y.
    
    Cosine similarity, or the cosine kernel, computes similarity as the
    normalized dot product of X and Y:
    
        K(X, Y) = <X, Y> / (||X||*||Y||)
    
    On L2-normalized data, this function is equivalent to linear_kernel.
    
    Read more in the :ref:`User Guide <cosine_similarity>`.
    
    Parameters
    ----------
    X : ndarray or sparse array, shape: (n_samples_X, n_features)
        Input data.
    
    Y : ndarray or sparse array, shape: (n_samples_Y, n_features)
        Input data. If ``None``, the output will be the pairwise
        similarities between all samples in ``X``.
    
    dense_output : boolean (optional), default True
        Whether to return dense output even when the input is sparse. If
        ``False``, the output is sparse if both input arrays are sparse.
    
 

#### Ranking: inefficient in memory and time (individually computing cosine similarity).

In [ ]:
# need to take [0][0]
cosine_similarity([[0, 1]], [[0, 1]])[0][0]

1.0

In [ ]:
trio.sort(key=lambda t: cosine_similarity([t[2]], [qv])[0][0], reverse=True)
trio

[(0, 'a', [0, 1]),
 (2, 'c', [0.2, 0.3]),
 (3, 'd', [0.5, 0.5]),
 (1, 'b', [1, 0])]

In [ ]:
def rank_candidates_basic(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    qv = question_to_vec(question=question, embeddings=embeddings, tokenizer=tokenizer)

    # record candidate representations, takes memory and space here
    candidates_v = [question_to_vec(c, embeddings, tokenizer) for c in candidates]

    # add the representations to the zipped unumerated candidates list
    # takes more space here, than if we just have enumerated list w/o representations
    trios = list(zip(range(len(candidates)), candidates, candidates_v))

    # sort by the cosine similarity
    trios.sort(key=lambda t: cosine_similarity([t[2]], [qv])[0][0], reverse=True)

    # drop the representations and return
    return [(t[0], t[1]) for t in trios]

#### Ranking: inefficient in time (individually computing cosine similarity), but less space used.

We do not need to pre-compute and store the representation of each candidate. Instead - compute them straight away for sorting.

In [ ]:
def rank_candidates_slow(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """

    # initial question representation
    # will reuse many times, so precompute
    qv = question_to_vec(question=question, embeddings=embeddings, tokenizer=tokenizer)

    # enumerated candidates
    enum_candidates = list(zip(range(len(candidates)), candidates))

    # sort by the cosine similarity between the representations of the
    # initial question and the representations of the candidate questions 
    # computed on the fly
    enum_candidates.sort(key=lambda x: cosine_similarity(
                                            [question_to_vec(x[1], embeddings, tokenizer)],
                                            [qv]
                                            )[0][0],
                        reverse=True) # want decreasing order of cosine similarity

    return [(ec[0], ec[1]) for ec in enum_candidates]

#### Ranking: better in time (compute cosine similarity for all of the candidates in a vectorised form), but worse in space

The 2 previous methods take 4-5 minutes for 1000 examples, this one takes ~1 minute

In [ ]:
# question and its vector representation
q = 'a'
qv = [0, 1]

# candidates and their vector representations
cs = ['a', 'b', 'c', 'd']
cvs = [[0, 1], [1, 0], [0.2, 0.3], [0.5, 0.5]]

print(np.array(qv).shape, np.array(cvs).shape)

(2,) (4, 2)


In [ ]:
dists = cosine_similarity([qv], cvs)
print(dists.shape, dists[0].shape)

trio = list(zip(range(len(cs)), cs, dists[0]))
trio.sort(key=lambda x: x[2], reverse=True)
trio

(1, 4) (4,)


[(0, 'a', 1.0),
 (2, 'c', 0.8320502943378436),
 (3, 'd', 0.7071067811865475),
 (1, 'b', 0.0)]

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """

    # question vector
    question_v = question_to_vec(question=question, embeddings=embeddings, tokenizer=tokenizer)
    # record candidate representations, takes memory and space here
    candidate_vs = [question_to_vec(c, embeddings, tokenizer) for c in candidates]

    # precomputing all together is faster (vectorized numpy operations)
    # (1, len(candidate_vs)), (len(candidate_vs),) -> (len(candidate_vs),)
    candidate_dists = cosine_similarity([question_v], candidate_vs)[0]

    # enumerated candidates
    enum_candidates = list(zip(range(len(candidates)),
                               candidates,
                               candidate_dists)
    )

    # sort by the cosine similarity between the representations of the
    # initial question and the representations of the candidate questions
    enum_candidates.sort(key=lambda x: x[2],
                         reverse=True) # want decreasing order of cosine similarity

    return [(ec[0], ec[1]) for ec in enum_candidates]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        print('Question:', question)

        ranks = rank_candidates(question=question, candidates=q_candidates,
                                embeddings=wv_embeddings, tokenizer=my_tokenizer)
        for rank in ranks:
            print(rank)
        print()

Question: converting string to list
(1, 'C# create cookie from string and send it')
(0, 'Convert Google results object (pure js) to Python object')
(2, 'How to use jQuery AJAX for an outside domain?')

Question: Sending array via Ajax fails
(0, 'Getting all list items of an unordered list in PHP')
(2, 'select2 not displaying search results')
(1, 'WPF- How to update the changes in list item of a list')



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [('*', 'Getting all list items of an unordered list in PHP'),        #скрыт
            ('*', 'select2 not displaying search results'),                     #скрыт
            ('*', 'WPF- How to update the changes in list item of a list')]]    #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

* Ответ: `021` если оспользовать .lower() в tokeniser-е, и `102` если использовать tokenizer из оригинальной версии этого jupyter notebook.

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, my_tokenizer)
    #print(q)
    #print(ranks)

    
    # incorrect, indexes are not recorded as strings
    # wv_ranking.append([r[0] for r in ranks].index('0') + 1)

    # Clear, but less efficient solution (writinfg in memory)
    #
    # all_ranks = [r[0] for r in ranks])
    # duplicate_index = all_ranks.index(0)   # duplicate is on the 0th position
    # duplicate_ rank = duplicate_index + 1  # ranks start from 1, unlike indexes
    # wv_ranking.append(duplicate_rank)
    #
    # the same, but all in one line
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

print('Done!')
wv_ranking[::50]

Done!


[5, 10, 124, 1, 1, 1, 664, 1, 1, 47, 1, 1, 66, 44, 16, 1, 5, 1, 1, 20]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.418 | Hits@   1: 0.418
DCG@   5: 0.505 | Hits@   5: 0.584
DCG@  10: 0.526 | Hits@  10: 0.652
DCG@ 100: 0.572 | Hits@ 100: 0.877
DCG@ 500: 0.585 | Hits@ 500: 0.974
DCG@1000: 0.588 | Hits@1000: 1.000



### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
# original train data
print("n pairs:", len(train_data), '\n')
train_data[:3]

n pairs: 1000000 



[['converting string to list',
  'Convert Google results object (pure js) to Python object\n'],
 ['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?',
  'Event handling for geometries in Three.js?\n'],
 ['Sending array via Ajax fails',
  'Getting all list items of an unordered list in PHP\n']]

In [ ]:
# read about word2vec:
# https://radimrehurek.com/gensim/models/word2vec.html

# glue the list of lists into the list of tuples (happens fast)
questions = [' '.join([l[0], l[1]]) for l in train_data]
questions[:3]

['converting string to list Convert Google results object (pure js) to Python object\n',
 'Which HTML 5 Canvas Javascript to use for making an interactive drawing tool? Event handling for geometries in Three.js?\n',
 'Sending array via Ajax fails Getting all list items of an unordered list in PHP\n']

In [ ]:
proc_words = [my_tokenizer.tokenize(q) for q in questions]
for q in proc_words[:5]:
    print(q)

['converting', 'string', 'list', 'convert', 'google', 'results', 'object', 'pure', 'js', 'python', 'object']
['html', '5', 'canvas', 'javascript', 'use', 'making', 'interactive', 'drawing', 'tool', 'event', 'handling', 'geometries', 'three', 'js']
['sending', 'array', 'via', 'ajax', 'fails', 'getting', 'list', 'items', 'unordered', 'list', 'php']
['insert', 'cookiecollection', 'cookiecontainer', 'c#', 'create', 'cookie', 'string', 'send']
['updating', 'one', 'element', 'bound', 'observable', 'collection', 'wpf', 'update', 'changes', 'list', 'item', 'list']


#### Choosing the window

In [ ]:
# for all questions give the number of tokens
all_num_words = list(map(lambda q: len(my_tokenizer.tokenize(q)),
                    questions))

quantiles_arr = np.quantile(all_num_words, [0, 0.25, 0.5, 0.75, 1])

In [ ]:
print("Mean number of tokens per question:",   np.mean(all_num_words))

quantiles = {
    'quantile_names': ['min', 'lq', 'median', 'uq', 'max'],
    'quantiles': quantiles_arr
    
}
pd.DataFrame(quantiles)

Mean number of tokens per question: 11.602786


,quantile_names,quantiles
0,min,1.0
1,lq,9.0
2,median,11.0
3,uq,13.0
4,max,61.0


Есть примеры с большим количеством слов на вопрос. 11-12 слов в среднем и по медиане на склеенное предложение, поэтому window=11 будет охватывать все предложение даже для слов в начале каждого предложения.

In [ ]:
from gensim.models import Word2Vec

In [ ]:
%%time
embeddings_trained = Word2Vec(
                proc_words,     # data for model to train on [[token1, t2, ..., t11], [t1, t2, ..., t10], [], []]
                size=200,       # embedding vector size
                min_count=5,    # consider words that occured at least 5 times
                window=11,      # central word, `window` words to the left and `window` to the right
                sg=1,           # use skip-grams instead of CBOW
                workers=4).wv 

CPU times: user 17min 26s, sys: 1.19 s, total: 17min 27s
Wall time: 9min


In [ ]:
embeddings_trained.index2word[:20]

['using',
 'android',
 'file',
 'java',
 'get',
 'error',
 'php',
 'use',
 'javascript',
 'data',
 'python',
 'string',
 'jquery',
 'function',
 'array',
 'value',
 'object',
 'c#',
 'net',
 'class']

In [ ]:
embeddings_trained

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    
    ranks = rank_candidates(q, ex, embeddings_trained, my_tokenizer)
    # ranks = rank_candidates(q, ex, wv_embeddings, my_tokenizer)

    if i % 50 == 0:
        print(f"Question {i}")
        print(q)
        print(ex)
        print(ranks)
        print()
        

    # wv_ranking.append([r[0] for r in ranks].index('0') + 1) # wrong, same as before
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

wv_ranking[::50]

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
wv_ranking[::50]

[1, 2, 190, 1, 1, 1, 11, 1, 1, 1, 1, 1, 98, 20, 11, 1, 1, 1, 2, 20]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.534 | Hits@   1: 0.534
DCG@   5: 0.621 | Hits@   5: 0.696
DCG@  10: 0.640 | Hits@  10: 0.754
DCG@ 100: 0.675 | Hits@ 100: 0.922
DCG@ 500: 0.684 | Hits@ 500: 0.987
DCG@1000: 0.685 | Hits@1000: 1.000



### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


1. **Токенизация.** Новый кастомный tokeniser работает лучше, чем оригинальный, который делал `regex match` на `'w+'`. DGC@1 растет с 0.28 до 0.41 Происходит это по нескольким причинам:
    * Новый tokeniser использует `.lower()`. Поэтому Python и python будут иметь одно и то же представление.
    * Новый tokenizer также выбирает символы `[+#]*`, конкретно эти символы были выбраны, чтобы C++ и C# оставались сами собой. Скорее всего, есть и другие символы, которые тоже хорошо бы учесть, но ничего путного в голову не пришло.
    * Из-за того, как мы делаем embedding предложения (берем среднее векторов и не учитываем последовательность), использовать пунктуацию не имеет смысла.
    * Дополнительно, мы убираем английские стоп-слова из `nltk`.

2. **Нормализация слов.** Лемматицация из spacy не помогла. Только замедлила процесс потому что пришлось делать следующее:
    * парсим с regex в лист и переводим в `.lower()` case.
    * соединяем через пробел, чтобы дать в spacy
    * берем `.lemma_` через list-comprehension и одновременно убираем стоп-слова

3. **Embeddings.** Натренированный embedding справлялся примерно так же как предобученный, когда я использовал парамертры как из семинара (DGC@1 = 0.41). Я изменил алгоритм с CBOW на skip-grams, и качество натренированного эмбеддинга подскочило до (DGC@1 = HITS@1 = 0.53). Но теперь тренировка стала занимать 17 минут, так что решил перестать пробовать - слишком много времени на каждый эксперимент, а уже нужно вторую домашку начинать :)

4. **Качество не блестящее**, но сносное (HITS@5 = 0.7). Улучшить скорее всего можно было бы с помощью более правильной комбинации regex для выбора нужных слов, может быть, что другая лемматизация дала бы качество лучше. В train set на самом деле не пары предложений, непонятно что делать, когда тройки и т.д. Зависит от того, похожие ли там вопросы или наоборот. Я решил брать только первые 2 вопроса. Это облегчает выбор window, так как количество tokens  равномернее в этом случае..

5. **Как улучшить:** Можно взять предобученный embedding и дообучить его на нашем train set.




## Remove data folder, but leave the shortcut.

In [ ]:
%rm -r ./data

In [ ]:
%ls

'Copy of [homework]simple_embeddings.ipynb'
'SO_vectors_200.bin?download=1'
 stackoverflow_similar_questions.zip
